## Load Data

In [ ]:
import pandas as pd

data = pd.read_csv("data.csv")
data = data["content"].drop_duplicates().dropna()[:37000]
print(data['content'].info())

## Removing Stopwords

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


stop_words = stopwords.words('english')

# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
data=data.apply(remove_stopwords)

## Lemmatization

In [29]:
import spacy
from gensim import corpora

# nlp = en_core_web_sm.load()
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output
from humanize import intword
tokenized_reviews = lemmatization(data.tolist())
print(intword(sum(len(x) for x in tokenized_reviews)), " Tokenizations")

dictionary = corpora.Dictionary(tokenized_reviews)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_reviews]

<class 'pandas.core.series.Series'>
RangeIndex: 50000 entries, 0 to 49999
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
50000 non-null  object
dtypes: object(1)
memory usage: 390.8+ KB
None


## Model Training

In [ ]:
import gensim
# Creating the object for LDA model using gensim library
Lda = gensim.models.LdaMulticore

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, id2word=dictionary, num_topics=25, passes=10)
print("-"*10, "\n", "Topics\n", ldamodel.print_topics(num_words=10))

## Evaluation

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
%matplotlib inline

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis

C:\Users\eslam\AppData\Roaming\Python\Python310\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
22    -0.122694 -0.041069       1        1  16.018410
23    -0.181438  0.013345       2        1   8.959833
20    -0.144445  0.092726       3        1   7.647688
12    -0.127424 -0.021320       4        1   6.853341
9     -0.060012 -0.164081       5        1   4.871896
24    -0.152159  0.014992       6        1   4.502978
10    -0.117213 -0.012677       7        1   4.204548
3     -0.183258  0.015314       8        1   4.188041
13    -0.050110  0.181965       9        1   3.826587
8     -0.056984  0.228259      10        1   3.805382
1      0.090208 -0.141644      11        1   3.780506
16     0.036433  0.001747      12        1   3.202194
6      0.059756 -0.253962      13        1   3.096560
2      0.079095 -0.148272      14        1   3.081791
19     0.072607 -0.106202      15        1   3.041933
21     0.021724 -0.112393      16        1   2.777116
15    -0.035642 -0.033460      17        1   2.766431
4     -0.075880  0.053824      18        1   2.412892
14     0.088047  0.215509      19        1   2.152296
18    -0.010647  0.011194      20        1   2.099729
17     0.007677 -0.035185      21        1   2.064753
0      0.163079 -0.010129      22        1   1.598130
11     0.130930  0.081579      23        1   1.350941
7      0.259344  0.071005      24        1   1.090512
5      0.309007  0.098934      25        1   0.605512, topic_info=         Term          Freq         Total Category  logprob  loglift
696     woman  21571.000000  21571.000000  Default  30.0000  30.0000
503    police  17728.000000  17728.000000  Default  29.0000  29.0000
107    people  63357.000000  63357.000000  Default  28.0000  28.0000
437        m.  11969.000000  11969.000000  Default  27.0000  27.0000
568    school  14101.000000  14101.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
80       late     15.125643   8483.192841  Topic25  -7.8159  -1.2226
653     today      4.296107   9483.732409  Topic25  -9.0746  -2.5928
6387   county      0.093674      1.173298  Topic25 -12.9002   2.5791
11024       %      0.093675      1.173303  Topic25 -12.9002   2.5791
7609   prayer      0.093674      1.173298  Topic25 -12.9002   2.5791

[1557 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1879      2  0.043099    .
1879      7  0.006830    .
1879      9  0.526727    .
1879     10  0.110927    .
1879     12  0.027555    .
...     ...       ...  ...
704       8  0.008014   ’s
704       9  0.017140   ’s
704      12  0.006381   ’s
704      18  0.013430   ’s
704      25  0.002671   ’s

[3954 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[23, 24, 21, 13, 10, 25, 11, 4, 14, 9, 2, 17, 7, 3, 20, 22, 16, 5, 15, 19, 18, 1, 12, 8, 6])

In [ ]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix,total_docs=len(data.index)))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.239377666616827

Coherence Score:  0.5377841348657011
